## 誤差逆伝播法に対応した２層ニューラルネットワーク

In [52]:
import sys, os
import numpy as np
from collections import OrderedDict
import math

sys.path.append("../deep-learning-from-scratch-master")
from dataset.mnist import load_mnist

### 関数の定義

In [53]:
# ソフトマックス関数
def softmax(x):
    """ミニバッチに対しても適用できるsoftmax関数"""
    x = x - np.max(x, axis=-1, keepdims=True)  # オーバーフロー対策
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)


# 交差エントロピー誤差
def cross_entropy_error(y, t):
    """予測値yと正解ラベルtの交差エントロピー誤差を返す。１次元のベクトルに対しても、行列（ミニバッチ）に対しても適用可。"""

    # バッチがない場合（yが１次元のベクトルの場合）も、1×n 行列に変換しておく
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換する
    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

### 各レイヤーの定義

In [54]:
# バッチ対応Affineレイヤー
class Affine:

    def __init__(self, W: np.ndarray, b: np.ndarray):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x: np.ndarray):
        self.x = x
        return np.dot(x, self.W) + self.b

    def backward(self, dout: np.ndarray):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        # dBはY1からYnまでの偏微分の和
        self.db = np.sum(dout, axis=0)
        return dx


# バッチ対応ReLUレイヤー
class ReLU:

    def __init__(self):
        self.mask = None

    def forward(self, x: np.ndarray):
        self.mask = x <= 0  # 行列の要素が0以下の場所を記録
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout: np.ndarray):
        dout[self.mask] = 0
        return dout


# バッチ対応Softmax-with-Lossレイヤー
class SoftmaxWithLoss:

    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x: np.ndarray, t: np.ndarray):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]  # type: ignore
        # バッチサイズで割り、データ１個あたりの誤差が伝播するようにしている
        dx = (self.y - self.t) / batch_size  # type: ignore
        return dx

### ２層ネットワークの定義

In [55]:
class TwoLayerNet:

    def __init__(
        self,
        inputSize: int,
        hiddenSize: int,
        outputSize: int,
        weightInitStd: float = 0.01,
    ) -> None:

        # 重みの初期化（W1, W2は標準偏差がweightInitStdの正規分布で初期化する）
        self.params = {}
        self.params["W1"] = weightInitStd * np.random.randn(inputSize, hiddenSize)
        self.params["b1"] = np.zeros(hiddenSize)
        self.params["W2"] = weightInitStd * np.random.randn(hiddenSize, outputSize)
        self.params["b2"] = np.zeros(outputSize)

        # レイヤーの生成
        self.layers = OrderedDict()  # 順番付きディクショナリを使用
        self.layers["Affine1"] = Affine(self.params["W1"], self.params["b1"])
        self.layers["Relu1"] = ReLU()
        self.layers["Affine2"] = Affine(self.params["W2"], self.params["b2"])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x: np.ndarray):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x: np.ndarray, t: np.ndarray):
        y = self.predict(x)
        # Softmax-with-Errorレイヤーで、教師ラベルとの交差エントロピー誤差を求める
        return self.lastLayer.forward(y, t)

    def accuracy(self, x: np.ndarray, t: np.ndarray):
        y = self.predict(x)
        y = np.argmax(y, axis=1)  # バッチに含まれる画像１枚ごとの、推測されたラベル

        if t.ndim != 1:  # 教師ラベルtがone-hot形式の場合は、ただの数字に変換する
            t = np.argmax(t, axis=1)

        return np.sum(y == t) / float(x.shape[0])

    def gradient(self, x: np.ndarray, t: np.ndarray):

        # 入力データをもとに推論し、交差エントロピー誤差を求める
        self.loss(x, t)

        # 逆伝播によってそれぞれの重み・バイアスの勾配を求める
        dout = self.lastLayer.backward(1)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 各レイヤーに記録された勾配の情報を記録する
        grads = {}
        grads["W1"] = self.layers["Affine1"].dW
        grads["b1"] = self.layers["Affine1"].db
        grads["W2"] = self.layers["Affine2"].dW
        grads["b2"] = self.layers["Affine2"].db

        return grads

### 誤差伝播法を使った学習

In [56]:
# MNISTデータの読み込み
(xTrain, tTrain), (xTest, tTest) = load_mnist(normalize=True, one_hot_label=True)

# ネットワークの初期化
hiddenSize = 50
network = TwoLayerNet(inputSize=784, hiddenSize=hiddenSize, outputSize=10)

itersNum = 10000
trainSize = xTrain.shape[0]
batchSize = 100
learningRate = 0.1

trainLossList = []
trainAccuracyList = []
testAccuracyList = []

iterPerEpoch = max(trainSize / batchSize, 1)

print("  EPOCH  | TEST_ACC | TRAIN_ACC ")
print("---------|----------|-----------")

# 学習を行う
for i in range(itersNum):

    # ランダムにバッチデータを選ぶ
    batchMask = np.random.choice(trainSize, batchSize)
    xBatch = xTrain[batchMask]
    tBatch = tTrain[batchMask]

    # 誤差逆伝播法によって勾配を求める
    grad = network.gradient(xBatch, tBatch)

    # 重みを更新する
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learningRate * grad[key]

    # バッチデータに対する損失関数を計算して記録する
    trainLossList.append(network.loss(xBatch, tBatch))

    # １エポックごとに、訓練データとテストデータに対する認識精度を記録する
    if i % iterPerEpoch == 0:
        trainAccuracy = network.accuracy(xTrain, tTrain)
        trainAccuracyList.append(trainAccuracy)
        testAccuracy = network.accuracy(xTest, tTest)
        testAccuracyList.append(testAccuracy)

        # 結果を表示する
        print(
            f'{"{: >3}".format(int(i / iterPerEpoch))} /{"{: >3}".format(math.floor(itersNum / iterPerEpoch))} | {trainAccuracy*100:7.3f}% | {testAccuracy*100:7.3f}%'
        )

  EPOCH  | TEST_ACC | TRAIN_ACC 
---------|----------|-----------
  0 / 16 |  10.238% |  10.730%
  1 / 16 |  90.673% |  90.850%
  2 / 16 |  92.332% |  92.610%
  3 / 16 |  93.727% |  93.590%
  4 / 16 |  94.540% |  94.410%
  5 / 16 |  95.263% |  95.000%
  6 / 16 |  95.672% |  95.380%
  7 / 16 |  96.032% |  95.540%
  8 / 16 |  96.438% |  95.940%
  9 / 16 |  96.733% |  96.240%
 10 / 16 |  96.980% |  96.330%
 11 / 16 |  97.147% |  96.320%
 12 / 16 |  97.372% |  96.840%
 13 / 16 |  97.420% |  96.590%
 14 / 16 |  97.768% |  96.960%
 15 / 16 |  97.780% |  96.920%
 16 / 16 |  97.815% |  96.840%
